## Test of conservative schemes
This is a one-dimensional simulation of Burger's equation using conservative schemes in the range $0 < x < 200$.

$$
\frac{\partial u}{\partial t} = u \frac{\partial u}{\partial x} 
$$

with initial conditions

$$
u(x, 0) = 1 + \epsilon \sin (k x)
$$

In [4]:
import numpy as np

In [5]:
# Grid
nx = 100
xl, xr = 0., 100.
nghost = 1

dx = (xr - xl) / nx
nxt = nx + 2 * nghost

# Courant number
nu = 0.8

# Advection speed (constant)
c = 1.

# Initial conditions for a small wave
eps = 0.1
L = 50

# Scheme
# 'lw', 'upwind'
scheme = 'upwind'

# Output dir
out_dir = 'c-upwind'

# Constant timestep
dt = nu * dx / c

# End time
t_end = 10.

# I/O
n_output = 4

# Initialize
t = 0.
nstep = 0


In [6]:
# Initialize grid

# Node points (Cell interfaces)
xn = np.linspace(xl-nghost*dx, xr+nghost*dx, nxt+1)

# Cell centered grid
xc = np.linspace(xl-0.5*nghost*dx, xr+0.5*nghost*dx, nxt)

In [7]:
# Initial conditions (small amplitude wave)
k = 2 * np.pi / L
u = 1 + eps * np.sin(k * xc)

In [9]:
# Time integration loop
while t < t_end:
    
    # Set Boundary conditions
    u[0] = u[1]
    u[-1] = u[-2]
    
    # Create left, right, and inner arrays to construct fluxes
    u_right = u[nghost+1:]
    u_left = u[:-nghost-1]
    u_inner = u[nghost:-nghost]

    # Right and left fluxes for Burger's equation
    fc_inner = u_inner * u_inner / 2.
    fc_right = u_right * u_right / 2.
    
    if scheme == 'upwind':
        fn = np.where(u_right + u_inner > 0, fc_inner, fc_right)
        fn_right = fn[1:] 
        fn_left = fn[:-1]
        
    elif scheme == 'lw':
        pass
        
    else:
        print('Error: Unknown scheme. Exiting.')
        exit()

    # Calculate new solution. One timestep in selected scheme.
    u_new_inner = u_inner - nu * (fn_right - fn_left)
    
    # Create a copy so that we can still use u_new_inner.
    u_inner = u_new_inner.copy()
    
    # Extend u to include ghost cells again
    u = np.concatenate(([0], u_inner, [0]))
    
    # Update time
    t = t + dt
    nstep = nstep + 1
    
    # I/O
    if nstep % n_output == 0:
        u_inner.tofile(f'{out_dir}/{scheme}-{int(nstep/n_output):0>4d}.dat')
        

ValueError: operands could not be broadcast together with shapes (100,) (99,) 